## Veiew DB in panadas

In [19]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results[['id', 'strategy_type', 'tab_name', 'ticker','trigger_price', 'strike_price', 'strike_buy','strike_sell','estimated_premium', 'last_price_when_checked',
         'timestamp_of_price_when_last_checked','scrape_date', 'timestamp_of_trigger','strategy_status', 'price_when_triggered', 'price_when_order_placed','premium_when_last_checked']].tail(25)

,id,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,scrape_date,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked
28,29,Bull Put,Mild Risk 95-97% accuracy > longer expiry,AAPL,216.82,sell 210.0 - buy 200.0,200.0,210.0,184.0,224.24,2025-03-27T19:01:22.570217,2025-03-27T08:52:55.221920,None,None,NaN,None,NaN
29,30,Bull Put,Mild Risk 95-97% accuracy > longer expiry,GOOGL,160.67,sell 155.0 - buy 145.0,145.0,155.0,180.0,162.51,2025-03-27T19:01:22.583084,2025-03-27T08:52:55.298004,None,None,NaN,None,NaN
30,31,Bull Put,Mild Risk 95-97% accuracy > longer expiry,MSFT,384.35,sell 365.0 - buy 355.0,355.0,365.0,114.0,391.24,2025-03-27T19:01:22.594396,2025-03-27T08:52:55.375810,None,None,NaN,None,NaN
31,32,Bull Put,Mild Risk 95-97% accuracy > longer expiry,QQQ,475.91,sell 455.0 - buy 445.0,445.0,455.0,160.0,483.06,2025-03-27T19:01:22.603242,2025-03-27T08:52:55.453856,None,None,NaN,None,NaN
32,33,Bull Put,Mild Risk 95-97% accuracy > longer expiry,WMT,83.2,sell 80.0 - buy 70.0,70.0,80.0,86.0,85.99,2025-03-27T19:01:22.611996,2025-03-27T08:52:55.528926,None,None,NaN,None,NaN
33,34,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,NKE,172,sell 60.0 - buy 50.0,50.0,60.0,74.0,66.00,2025-03-27T19:01:22.635997,2025-03-27T08:52:59.081604,2025-03-27T09:50:04.362659,premium too low,65.65,None,0.0
34,35,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TSM,166.68,sell 160.0 - buy 150.0,150.0,160.0,238.0,168.56,2025-03-27T19:01:22.646819,2025-03-27T08:52:59.158309,None,None,NaN,None,NaN
35,36,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,MSFT,402.27,sell 420.0 - buy 430.0,430.0,420.0,93.0,389.88,2025-03-28T06:44:55.091588,2025-03-28T06:08:03.723676,None,None,NaN,None,NaN
36,37,Bear Call,Minimal Risk 97-99% accuracy > shorter expiry,None,None,None,0.0,0.0,None,NaN,None,2025-03-28T06:08:07.296691,None,None,NaN,None,NaN
37,38,Bear Call,Mild Risk 95-97% accuracy > longer expiry,SQQQ,36.42,sell 38.0 - buy 45.0,45.0,38.0,155.0,35.55,2025-03-28T06:44:55.075722,2025-03-28T06:08:10.858267,None,None,NaN,None,NaN


In [15]:
results.columns

Index(['id', 'scrape_date', 'strategy_type', 'tab_name', 'ticker',
       'trigger_price', 'strike_price', 'strike_buy', 'strike_sell',
       'estimated_premium', 'last_price_when_checked',
       'timestamp_of_price_when_last_checked', 'item_id',
       'options_expiry_date', 'date_info', 'timestamp_of_trigger',
       'strategy_status', 'price_when_triggered', 'price_when_order_placed',
       'premium_when_last_checked', 'timestamp_of_order'],
      dtype='object')

In [18]:
results[~results['timestamp_of_trigger'].isnull()]

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received
8,9,2025-03-10T08:41:21.213768,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,WMT,89.68,sell 86.0 - buy 75.0,75.0,86.0,58.0,4,2025-03-21,March,2025-03-10T14:51:27.476449,premium too low,87.97,NaN
12,13,2025-03-10T08:41:28.351534,Bull Put,Mild Risk 95-97% accuracy > longer expiry,META,589.67,sell 540.0 - buy 530.0,530.0,540.0,97.0,2,2025-03-28,March,2025-03-10T14:57:11.292093,premium too low,589.12,NaN
14,15,2025-03-10T08:41:28.538762,Bull Put,Mild Risk 95-97% accuracy > longer expiry,WMT,89.29,sell 86.0 - buy 80.0,80.0,86.0,70.0,4,2025-03-28,March,2025-03-10T14:51:25.356915,premium too low,87.97,NaN
16,17,2025-03-10T08:41:32.218233,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,QQQ,490,sell 460.0 - buy 450.0,450.0,460.0,273.0,2,2025-03-28,March,2025-03-10T08:46:27.463750,premium too low,488.20,NaN
17,18,2025-03-10T08:41:32.354756,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TQQQ,62.25,sell 60.0 - buy 50.0,50.0,60.0,210.0,3,2025-03-28,March,2025-03-10T14:33:41.525971,premium too low,61.69,NaN
46,47,2025-03-12T06:02:01.555882,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,JNJ,163.62,sell 157.5 - buy 148.0,148.0,157.5,72.0,1,2025-03-28,March,2025-03-12T21:19:42.678459,order placed,162.70,60.0


## Delete ALL rows in SQLite database

In [32]:

import sqlite3

conn = sqlite3.connect('option_strategies.db')
c = conn.cursor()

# Delete all rows from table
c.execute('DELETE FROM option_strategies;',)

print('We have deleted', c.rowcount, 'records from the table.')

# Commit the changes to db			
conn.commit()
# Close the connection
conn.close()

We have deleted 18 records from the table.


## Delete table

In [3]:

# importing sqlite module 
import sqlite3 
  
# create connection to the  
# database geek 
connection = sqlite3.connect('option_strategies.db') 
  
# drop table 
connection.execute("DROP TABLE option_strategies") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully


## Update Row

In [14]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (390, 52))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## Add New column

In [13]:
import sqlite3
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def add_premium_column():
    """Add a 'timestamp_of_order ' column to the option_strategies table."""
    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Check if the column already exists
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [col[1] for col in cursor.fetchall()]
        
        if 'timestamp_of_order ' in columns:
            logger.info("Column 'timestamp_of_order  already exists in the table.")
            return
        
        # Add the new column - using REAL type for storing decimal/float values
        cursor.execute("""
            ALTER TABLE option_strategies 
            ADD COLUMN timestamp_of_order DATETIME
        """)
        
        conn.commit()
        logger.info("Successfully added 'timestamp_of_order' column to option_strategies table.")
        
    except sqlite3.Error as e:
        logger.error(f"SQLite error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    add_premium_column()

2025-03-27 10:06:28,587 - INFO - Successfully added 'timestamp_of_order' column to option_strategies table.
